In [1]:
# carregar llibreries
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [2]:
# Definir la url de la web que es desitge scrapejar
url_base = "https://www.bdfutbol.com/a/j__baleares.html"

# Fer la solicitud
respuesta = requests.get(url_base)
respuesta.status_code

200

In [3]:
# Extraure el HTML
html = respuesta.content

# Convertir HTML a BeautifulSoup object. 
soup = BeautifulSoup(html, "html.parser")

In [4]:
# cerquem les taules
tables = soup.find_all("table", {"id": "taul"})
tables

[<table class="taula_estil sortable taulaagrupats autonumeric-first-col" id="taul">
 <tr>
 <th></th>
 <th class="text-left">Àlies</th>
 <th class="text-left resp_off">Nom</th>
 <th class="text-left resp_off">Data de naixement</th>
 <th class="text-left poble resp_off">Lloc de naixement</th>
 <th title="Demarcació">Pos</th>
 <th title="Temporades">T</th>
 <th title="Partits Jugats">PJ</th>
 <th class="resp_off" title="Targetes Grogues">TG</th>
 <th class="resp_off" title="Targetes Vermelles">TV</th>
 <th title="Gols">G</th>
 </tr>
 <tr>
 <td>1</td>
 <td class="text-left"><a href="../j/j47.html">Albert Cano</a></td>
 <td class="text-left resp_off">Albert Cano López</td>
 <td class="text-left resp_off">01/08/1983</td>
 <td class="text-left poble resp_off">Palma</td>
 <td><div class="mig"></div></td>
 <td>7</td>
 <td>71</td>
 <td class="resp_off">20</td>
 <td class="resp_off">2</td>
 <td>3</td>
 <tr>
 <td>2</td>
 <td class="text-left"><a href="../j/j56.html">Riera</a></td>
 <td class="text

In [5]:
# descarreguem les dades resum de tots els jugadors

headers_lst = []
player_ids = []
urls= []
values_lst = []
pos = []

for table in tables:
    
    headers = table.find_all('th',{'class':'text-left'})
    for header in headers:
        headers_lst.append(header.text)
        
    dem = table.find_all('th',{'title':'Demarcació'})
    temp = table.find_all('th', {'title': 'Temporades'})
    PJ = table.find_all('th', {'title': 'Partits Jugats'})
    TG = table.find_all('th', {'title': 'Targetes Grogues'})
    TV = table.find_all('th', {'title': 'Targetes Vermelles'})
    G = table.find_all('th', {'title': 'Gols'})
    
    headers2 = [dem,temp,PJ,TG,TV,G]
    for header in headers2:
        headers_lst.append(header[0].text)
        
    tds = table.find_all('td')
    
    idx = 0 # creació id únic per jugador
    for td in tds:
        try:
            if 'class' in td.div.attrs:
                pos.append([td.div.attrs['class'][0],idx])
                idx=idx+1
            
        except:
            pass
        
    values_lst.append([None if td.text is None else  td.text for td in tds])

    refs = table.find_all("a")
    idx = 0
    for ref in refs:
        urls.append('https://www.bdfutbol.com'+ref.get('href')[2:])
        player_ids.append(idx)
        idx=idx+1

url_d = dict(zip(player_ids, urls))

headers_lst.insert(0,'idx')

In [6]:
# subdividim la llista obtinguda
player_info_lst = [values_lst[0][x:x+11] for x in range(0, len(values_lst[0]), 11)]

In [7]:
# cream dataframe
gen_df = pd.DataFrame (player_info_lst, columns = headers_lst)
gen_df

,idx,Àlies,Nom,Data de naixement,Lloc de naixement,Pos,T,PJ,TG,TV,G
0,1,Albert Cano,Albert Cano López,01/08/1983,Palma,,7,71,20,2,3
1,2,Riera,Albert Riera Ortega,15/04/1982,Manacor,,8,178,28,3,26
2,3,Prats,Antonio Prats Cervera,09/09/1971,Capdepera,,18,479,45,5,1
3,4,Tuni,Antoni Lluis Adrover Colom,14/06/1982,Sóller,,11,262,30,0,24
4,5,Toni Muñoz,Antonio Muñoz Llompart,13/08/1982,Palma,,3,34,1,0,2
...,...,...,...,...,...,...,...,...,...,...,...
860,861,Ramón,Gabriel Ramón Florit,08/03/1964,Palma,,9,248,30,1,60
861,862,Colomer,Miguel Colomer Nadal,07/07/1924,Manacor,,3,22,0,0,4
862,863,Brondo,Jaime Brondo Ferrer,16/09/1922,Palma,,5,38,0,2,7
863,864,Planas,Juan Planas Ferragut,10/11/1957,Llubí,,1,19,2,0,0


In [8]:
# insertam la posició si coincideix amb idx
pos_v = []
pos_i = []
for ttuple in iter(pos):
    pos_i.append(ttuple[1])
    pos_v.append(ttuple[0])
    
gen_df.iloc[pos_i,5] = pos_v
gen_df

,idx,Àlies,Nom,Data de naixement,Lloc de naixement,Pos,T,PJ,TG,TV,G
0,1,Albert Cano,Albert Cano López,01/08/1983,Palma,mig,7,71,20,2,3
1,2,Riera,Albert Riera Ortega,15/04/1982,Manacor,mig,8,178,28,3,26
2,3,Prats,Antonio Prats Cervera,09/09/1971,Capdepera,por,18,479,45,5,1
3,4,Tuni,Antoni Lluis Adrover Colom,14/06/1982,Sóller,mig,11,262,30,0,24
4,5,Toni Muñoz,Antonio Muñoz Llompart,13/08/1982,Palma,dav,3,34,1,0,2
...,...,...,...,...,...,...,...,...,...,...,...
860,861,Ramón,Gabriel Ramón Florit,08/03/1964,Palma,dav,9,248,30,1,60
861,862,Colomer,Miguel Colomer Nadal,07/07/1924,Manacor,mig,3,22,0,0,4
862,863,Brondo,Jaime Brondo Ferrer,16/09/1922,Palma,mig,5,38,0,2,7
863,864,Planas,Juan Planas Ferragut,10/11/1957,Llubí,def,1,19,2,0,0


In [11]:
# Scrapejem la info de cada temporada per tots els jugadors

dfs = pd.DataFrame()

for idx,url in url_d.items():
    
    # Definir la url de la web que es desitge scrapear
    url_player = url

    # Fer la solicitud
    respuesta = requests.get(url_player)
    respuesta.status_code
    soup = BeautifulSoup(respuesta.content, 'html.parser')

    tables = soup.find_all("table", {"class": "taula_estil"})

    divs_lst = []
    edats_lst = []
    PJs_lst = []
    PTs_lst = []
    PCs_lst = []
    PSs_lst = []
    MINs_lst = []
    TGs_lst = []
    TVs_lst = []
    Gs_lst = []
    idx_lst = []
    names_lst = []
    alies_lst = []
    temps_lst = []
    teams_lst = []

    for table in tables:


        temps = table.find_all('td',{'class':'colTemporada'})
        for temp in temps:
            temps_lst.append(temp.text.split('\n')[1])
                
        teams = table.find_all('td',{'class': 'colEquip'})
        for team in teams:
            teams_lst.append(team.text.split('\n')[2])
        divs = table.find_all('td', {'class': 'fit colDivisio'})
        for div in divs:
            divs_lst.append(div.text)
            idx_lst.append(idx)
            alies_lst.append(gen_df.iloc[idx]['Àlies'])
            names_lst.append(gen_df.iloc[idx]['Nom'])
        edats = table.find_all('td', {'class': 'fit colEdat vora-right'})
        for pj in edats:
            edats_lst.append(pj.text)
        PJs = table.find_all('td', {'class': 'fit colPJ'})
        for div in PJs:
                PJs_lst.append(div.text)
        PTs = table.find_all('td', {'class': 'fit colPT resp_off'})
        for div in PTs:
            PTs_lst.append(div.text)
        PCs = table.find_all('td', {'class': 'fit colPC resp_off'})
        for div in PCs:
            PCs_lst.append(div.text)
        PSs = table.find_all('td', {'class': 'fit colPS resp_off'})
        for div in PSs:
            PSs_lst.append(div.text)
        MINs = table.find_all('td', {'class': 'fit colMin resp_off'})
        for div in MINs:
            MINs_lst.append(div.text)
        TGs = table.find_all('td', {'class': 'fit colTG resp_off'})
        for div in TGs:
            TGs_lst.append(div.text)
        TVs = table.find_all('td', {'class': 'fit colTV resp_off'})
        for div in TVs:
            TVs_lst.append(div.text)
        Gs = table.find_all('td', {'class': 'fit colGols'})
        for div in Gs:
            Gs_lst.append(div.text)
            
    df = pd.DataFrame({
        'idx': idx_lst,
        'alies': alies_lst,
        'nom' : names_lst,
        'equip': teams_lst,
        'temp': temps_lst,
        'div': divs_lst,
        'edat': edats_lst,
        'PJ': PJs_lst[:-1],
        'PT': PTs_lst,
        'PC': PCs_lst,
        'PS': PSs_lst,
        'MIN': MINs_lst,
        'TG': TGs_lst,
        'TV': TVs_lst,
        'G': Gs_lst[:-1]
    })
    dfs = pd.concat([dfs, df])

In [12]:
dfs

,idx,alies,nom,equip,temp,div,edat,PJ,PT,PC,PS,MIN,TG,TV,G
0,0,Albert Cano,Albert Cano López,Mallorca B,2002-03,2ªB,19,15,6,3,9,551,2,0,1
1,0,Albert Cano,Albert Cano López,Mallorca B,2003-04,2ªB,20,15,8,5,7,707,2,1,0
2,0,Albert Cano,Albert Cano López,Albacete,2004-05,1ª,21,2,1,1,1,118,0,0,0
3,0,Albert Cano,Albert Cano López,Albacete,2005-06,2ª,22,0,0,0,0,0,0,0,0
4,0,Albert Cano,Albert Cano López,Almansa,2005-06,2ªB,22,7,5,4,2,471,4,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1,863,Planas,Juan Planas Ferragut,Constancia,1977-78,,20,-,-,-,-,-,-,-,-
2,863,Planas,Juan Planas Ferragut,Constancia,1979-80,,22,-,-,-,-,-,-,-,-
3,863,Planas,Juan Planas Ferragut,Poblense,1980-81,,23,-,-,-,-,-,-,-,-
4,863,Planas,Juan Planas Ferragut,Poblense,1982-83,2ªB,25,19,10,5,9,1007,2,0,0


In [13]:
dfs.to_csv('player_stats.csv')

In [14]:
gen_df.to_csv('players_summary.csv')